In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)

file = 'sensitivity_10_1e5.csv'

def mad(df):
    return (df - df.median()).abs().median()

In [2]:
sens = pd.DataFrame(np.loadtxt(f'../out/{file}'), columns=['r1', 'r2', 'r3', 't1', 't2', 't3', 'det', 'det_true'])
sens = sens.groupby(['r1', 'r2', 'r3', 't1', 't2', 't3']).agg([mad, np.median]).reset_index()
opt = sens[(sens.t1 == sens.r1) & (sens.t2 == sens.r2) & (sens.t3 == sens.r3)]\
            .filter([('t1', ''), ('t2', ''), ('t3', ''), ('det_true', 'median')])
opt = opt.rename(columns={'det_true': 'det_opt'})

sens = sens.merge(opt, on=[('t1', ''), ('t2', ''), ('t3', '')])
sens['ratio'] = sens[('det_true', 'mad')] / sens[('det_opt', 'median')]
sens

r1    r2    r3    t1    t2    t3       det             det_true  \
                                              mad     median       mad   
0     0.1   0.1   0.1   0.1   0.1   0.1  0.000343  11.488274  0.000343   
1     0.1   0.1   1.0   0.1   0.1   0.1  0.002446   9.317828  0.003726   
2     0.1   0.1  10.0   0.1   0.1   0.1  0.000582   6.845921  0.003048   
3     0.1   1.0   0.1   0.1   0.1   0.1  0.001211   8.447628  0.000604   
4     0.1   1.0   1.0   0.1   0.1   0.1  0.000993   7.488980  0.001137   
..    ...   ...   ...   ...   ...   ...       ...        ...       ...   
724  10.0   1.0   1.0  10.0  10.0  10.0  0.002571   2.803985  0.002462   
725  10.0   1.0  10.0  10.0  10.0  10.0  0.000789   2.351205  0.001862   
726  10.0  10.0   0.1  10.0  10.0  10.0  0.000474   2.249044  0.000591   
727  10.0  10.0   1.0  10.0  10.0  10.0  0.000443   2.210031  0.000512   
728  10.0  10.0  10.0  10.0  10.0  10.0  0.000281   1.981430  0.000281   

                  det_opt     ratio  
        median     median            
0    11.488274  11.488274  0.000030  
1    11.481080  11.488274  0.000324  
2    11.351051  11.488274  0.000265  
3    11.448078  11.488274  0.000053  
4    11.449215  11.488274  0.000099  
..         ...        ...       ...  
724   1.971439   1.981430  0.001242  
725   1.971956   1.981430  0.000940  
726   1.981268   1.981430  0.000298  
727   1.980599   1.981430  0.000258  
728   1.981430   1.981430  0.000142  

[729 rows x 12 columns]

In [3]:
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.subplots import make_subplots
import re

fig = make_subplots(3, 3, subplot_titles=[
    f'$r_4 = {r3} \quad \eta = {eta}$' for r3 in [0.1, 1, 10] for eta in [0.1, 1, 10]
], horizontal_spacing=0.05, vertical_spacing=0.1)
for j, x in enumerate([0.1, 1, 10]):
    sens1 = sens[(sens.t1 == sens.t2) & (sens.t2 == sens.t3) & (sens.t1 == x)]
    for i, (r3, d1) in enumerate(sens1.groupby('r3')):
        z = d1.sort_values(['r2', 'r1']).ratio.to_numpy().reshape(3, 3)
        z = np.flipud(z)
        fig.add_trace(go.Heatmap(
            z=z*100,
            x=['0.1', '1', '10'],
            y=['10', '1', '0.1'],
            coloraxis='coloraxis',
            text=z*100,
            texttemplate='%{text:.2f}%',
            textfont={'size': 8}
        ), row=i+1, col=j+1)
fig.update_layout(
    coloraxis={'colorscale': 'viridis'},
    height=750,
)
fig.update_xaxes(title='$r_2$', row=3)
fig.update_yaxes(title='$r_3$', col=1)
fig.show()
fig.write_image('../figures/sensitivity_mad.png')

In [39]:
import plotly.express as px
from functools import reduce
from plotly.colors import DEFAULT_PLOTLY_COLORS

fig = go.Figure()
for i, r in enumerate([1, 2, 3]):
    cond = reduce(lambda x, y: x & y, [sens[f'r{rx}'] == sens.t1 for rx in [1, 2, 3] if rx != r])
    sens_temp = sens[(sens.t1 == sens.t2) & (sens.t2 == sens.t3) & cond]
    color = DEFAULT_PLOTLY_COLORS[i].split('(')[1].split(')')[0].split(', ')

    for j, eta in enumerate([0.1, 1, 10]):
        sens_eta = sens_temp[sens_temp.t1 == eta].sort_values(f'r{r}')
        fig.add_trace(go.Scatter(
            x=sens_eta[f'r{r}'], y=sens_eta.ratio, legendgroup=eta, legendgrouptitle_text=f'$\eta = {eta}$',
            name=f'$r_{r}$', line_color=f'rgba({", ".join(color)}, {(j+1)/3})'
        ))

fig.update_xaxes(type='log')
fig.update_layout(legend=dict(groupclick="toggleitem"))
fig.show()
# fig.write_image('r2.png')